In [1]:
import numpy as np
import pandas as pd
import os
import random

def set_seed(seed: int):
    random.seed(seed) # Python
    np.random.seed(seed)  # Numpy, é o gerador utilizado pelo sklearn
    os.environ["PYTHONHASHSEED"] = str(seed)  # sistema operativo

set_seed(25)

# Making our own rnn

In [2]:
import numpy as np
import pandas as pd
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Dropout, Input, Embedding
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split

# Parameters
max_features = 10000  # Tamanho do vocabulário
maxlen = 120  # Tamanho máximo das sequências

# Load dos dados
csv_path = '../../datasets/human_or_ai_dataset_small.csv'  # Change this to your file path
df = pd.read_csv(csv_path)
# Sanity check!
print("Dataset shape:", df.shape)
print("Columns:", df.columns)

Dataset shape: (5051, 2)
Columns: Index(['text', 'source'], dtype='object')


In [3]:
# Separar os textos das labels
texts = df['text'].values
labels = df['source'].values

# Criar um tokenizer
tokenizer = preprocessing.text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(texts)

# Converter os textos para sequências de inteiros
sequences = tokenizer.texts_to_sequences(texts)

# Padding para uniformizar tamanhos
x_data = preprocessing.sequence.pad_sequences(sequences, maxlen=maxlen)

# Garantir label numérica
if not np.issubdtype(labels.dtype, np.number):
    label_map = {'human': 0, 'ai': 1}
    y_data = np.array([label_map[label] for label in labels])
else:
    y_data = labels
print(y_data)
# Data split !
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)

# Check shapes
print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)
print("x_test shape:", x_test.shape)
print("y_test shape:", y_test.shape)

# Print a sample
print("Sample sequence:", x_train[0])
print("Sample label:", y_train[0])

[0 1 0 ... 0 0 0]
x_train shape: (4040, 120)
y_train shape: (4040,)
x_test shape: (1011, 120)
y_test shape: (1011,)
Sample sequence: [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0   14   19    1  738    2 4758
  109 3150    5    1  619 1381  564    5    4  220  428    2  187   45
 2567  724 3261  111   14 1026    1 3273  482 1822   22 5021    5   38
  111    3   57    8  109    7  244    9    1  623  158  803 7358 3261
   44   14  143    8 1822 7520   96    1  531  373    7  130 3668   14
   57    8  109    7   30  244    5    4 6789   44   10   23  433  472
    6  266   22 1935    5  531   45   66]
Sample label: 0


## Define and train the model

In [4]:
model = Sequential()

model.add(Input((maxlen,))) 
model.add(Embedding(input_dim=max_features, output_dim=128, input_length=maxlen))
model.add(SimpleRNN(128, dropout=0.2, recurrent_dropout=0.2))  
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))  

model.summary()

/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 120, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,321,217 (5.04 MB)

 Trainable params: 1,321,217 (5.04 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
# Compilar o modelo
model.compile(
    optimizer='adam',  
    loss='binary_crossentropy',  
    metrics=['accuracy']
)

# Early stopping 
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=3,
    restore_best_weights=True
)

# Salvar o melhor modelo
model_checkpoint = ModelCheckpoint(
    'best_model_rnn.h5',
    monitor='val_accuracy',
    save_best_only=True
)

# Treinar o modelo com callbacks
history = model.fit(
    x_train, y_train,
    epochs=10, 
    batch_size=128,
    validation_split=0.2,
    callbacks=[early_stopping, model_checkpoint]
)

# Avaliar no conjunto de teste
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Accuracy no teste: {test_acc:.4f}")

Epoch 1/10
25/26 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.5045 - loss: 0.7075

26/26 ━━━━━━━━━━━━━━━━━━━━ 3s 86ms/step - accuracy: 0.5051 - loss: 0.7074 - val_accuracy: 0.5359 - val_loss: 0.6861
Epoch 2/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 77ms/step - accuracy: 0.5326 - loss: 0.7010 - val_accuracy: 0.5198 - val_loss: 0.6865
Epoch 3/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 81ms/step - accuracy: 0.5497 - loss: 0.6954 - val_accuracy: 0.5198 - val_loss: 0.6848
Epoch 4/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 81ms/step - accuracy: 0.5453 - loss: 0.6937 - val_accuracy: 0.5198 - val_loss: 0.6786
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5850 - loss: 0.6819
Accuracy no teste: 0.5776


### LSMT

In [6]:
from tensorflow.keras.layers import LSTM

model = Sequential()

model.add(Input((maxlen,))) 
model.add(Embedding(input_dim=max_features, output_dim=128, input_length=maxlen))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))  
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))  

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 120, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,419,905 (5.42 MB)

 Trainable params: 1,419,905 (5.42 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
# Compilar o modelo
model.compile(
    optimizer='adam',  
    loss='binary_crossentropy',  
    metrics=['accuracy']
)

# Early stopping 
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=3,
    restore_best_weights=True
)

# Salvar o melhor modelo
model_checkpoint = ModelCheckpoint(
    'best_model_rnn_lstm.h5',
    monitor='val_accuracy',
    save_best_only=True
)

# Treinar o modelo com callbacks
history = model.fit(
    x_train, y_train,
    epochs=10, 
    batch_size=128,
    validation_split=0.2,
    callbacks=[early_stopping, model_checkpoint]
)

# Avaliar no conjunto de teste
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Accuracy no teste: {test_acc:.4f}")

Epoch 1/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 305ms/step - accuracy: 0.5925 - loss: 0.6431

26/26 ━━━━━━━━━━━━━━━━━━━━ 10s 340ms/step - accuracy: 0.5969 - loss: 0.6397 - val_accuracy: 0.8886 - val_loss: 0.3021
Epoch 2/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 309ms/step - accuracy: 0.9173 - loss: 0.2248

26/26 ━━━━━━━━━━━━━━━━━━━━ 9s 335ms/step - accuracy: 0.9178 - loss: 0.2235 - val_accuracy: 0.9208 - val_loss: 0.1950
Epoch 3/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 300ms/step - accuracy: 0.9543 - loss: 0.1622

26/26 ━━━━━━━━━━━━━━━━━━━━ 9s 327ms/step - accuracy: 0.9543 - loss: 0.1616 - val_accuracy: 0.9220 - val_loss: 0.1780
Epoch 4/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 301ms/step - accuracy: 0.9804 - loss: 0.0549

26/26 ━━━━━━━━━━━━━━━━━━━━ 9s 329ms/step - accuracy: 0.9805 - loss: 0.0549 - val_accuracy: 0.9245 - val_loss: 0.1942
Epoch 5/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 295ms/step - accuracy: 0.9885 - loss: 0.0369

26/26 ━━━━━━━━━━━━━━━━━━━━ 8s 325ms/step - accuracy: 0.9885 - loss: 0.0368 - val_accuracy: 0.9443 - val_loss: 0.2475
Epoch 6/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 9s 339ms/step - accuracy: 0.9900 - loss: 0.0322 - val_accuracy: 0.9307 - val_loss: 0.2557
Epoch 7/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 9s 336ms/step - accuracy: 0.9939 - loss: 0.0273 - val_accuracy: 0.9344 - val_loss: 0.2860
Epoch 8/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 9s 335ms/step - accuracy: 0.9957 - loss: 0.0136 - val_accuracy: 0.9208 - val_loss: 0.2573
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - accuracy: 0.9392 - loss: 0.2547
Accuracy no teste: 0.9347


### GRU

In [8]:
from tensorflow.keras.layers import GRU

model = Sequential()

model.add(Input((maxlen,))) 
model.add(Embedding(input_dim=max_features, output_dim=128, input_length=maxlen))
model.add(GRU(128, dropout=0.2, recurrent_dropout=0.2))  
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))  

model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (None, 120, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ (None, 128)            │        99,072 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,387,393 (5.29 MB)

 Trainable params: 1,387,393 (5.29 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
# Compilar o modelo
model.compile(
    optimizer='adam',  
    loss='binary_crossentropy',  
    metrics=['accuracy']
)

# Early stopping 
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=3,
    restore_best_weights=True
)

# Salvar o melhor modelo
model_checkpoint = ModelCheckpoint(
    'best_model_rnn_gru.h5',
    monitor='val_accuracy',
    save_best_only=True
)

# Treinar o modelo com callbacks
history = model.fit(
    x_train, y_train,
    epochs=10, 
    batch_size=128,
    validation_split=0.2,
    callbacks=[early_stopping, model_checkpoint]
)

# Avaliar no conjunto de teste
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Accuracy no teste: {test_acc:.4f}")

Epoch 1/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 252ms/step - accuracy: 0.5602 - loss: 0.6786

26/26 ━━━━━━━━━━━━━━━━━━━━ 9s 280ms/step - accuracy: 0.5621 - loss: 0.6774 - val_accuracy: 0.8045 - val_loss: 0.4411
Epoch 2/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 264ms/step - accuracy: 0.8732 - loss: 0.3606

26/26 ━━━━━━━━━━━━━━━━━━━━ 7s 286ms/step - accuracy: 0.8737 - loss: 0.3582 - val_accuracy: 0.8998 - val_loss: 0.2451
Epoch 3/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 7s 282ms/step - accuracy: 0.9536 - loss: 0.1379 - val_accuracy: 0.8973 - val_loss: 0.2385
Epoch 4/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 261ms/step - accuracy: 0.9719 - loss: 0.0863

26/26 ━━━━━━━━━━━━━━━━━━━━ 7s 285ms/step - accuracy: 0.9721 - loss: 0.0859 - val_accuracy: 0.9208 - val_loss: 0.2358
Epoch 5/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 7s 284ms/step - accuracy: 0.9903 - loss: 0.0348 - val_accuracy: 0.8738 - val_loss: 0.3254
Epoch 6/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 7s 286ms/step - accuracy: 0.9884 - loss: 0.0420 - val_accuracy: 0.9171 - val_loss: 0.3422
Epoch 7/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 7s 284ms/step - accuracy: 0.9947 - loss: 0.0186 - val_accuracy: 0.8948 - val_loss: 0.3924
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.9265 - loss: 0.2189
Accuracy no teste: 0.9159


# Predicting for the Competition

In [11]:
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model('best_model_rnn_gru.h5')

print("Model loaded successfully!")

# Carregar o CSV com dados para prever
prediction_csv_path = '../../datasets/dataset1_inputs.csv'
df_predict = pd.read_csv(prediction_csv_path,sep="\t")

# Verificar os dados carregados
print("Prediction dataset shape:", df_predict.shape)
print("Columns:", df_predict.columns)
print("Sample IDs:", df_predict['ID'].head())

# Pré-processar os dados de texto para corresponder ao formato de treinamento
# Converter textos para sequências
sequences = tokenizer.texts_to_sequences(df_predict['Text'].values)

# Padding das sequências para o mesmo tamanho usado no treinamento
x_predict = preprocessing.sequence.pad_sequences(sequences, maxlen=maxlen)

# Fazer previsões com o modelo treinado
predictions = model.predict(x_predict)

# Converter probabilidades para labels binários (0 = Human, 1 = AI)
# Usando 0.5 como threshold - você pode ajustar isso conforme necessário
labels = (predictions > 0.5).astype(int)

# Mapear labels para "AI" e "Human"
label_mapping = {1: "AI", 0: "Human"}
labels_mapped = [label_mapping[label] for label in labels.flatten()]

# Criar um DataFrame com os resultados
results_df = pd.DataFrame({
    'ID': df_predict['ID'],
    'Label': labels_mapped
})

# Exibir uma amostra dos resultados
print("\nAmostra dos resultados de previsão:")
print(results_df.head())

# Salvar em CSV
output_csv_path = 'prediction_results3.csv'
results_df.to_csv(output_csv_path, sep="\t", index=False)
print(f"\nResultados salvos em {output_csv_path}")

Model loaded successfully!
Prediction dataset shape: (30, 2)
Columns: Index(['ID', 'Text'], dtype='object')
Sample IDs: 0    D1-1
1    D1-2
2    D1-3
3    D1-4
4    D1-5
Name: ID, dtype: object
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step

Amostra dos resultados de previsão:
     ID  Label
0  D1-1  Human
1  D1-2  Human
2  D1-3  Human
3  D1-4  Human
4  D1-5     AI

Resultados salvos em prediction_results3.csv
